# <center>MDS6212 Fintech Theory and Practice: Week 3 Assignment<center>

----
    
**220041006 *Yihang Li***

**Data confidential requirements:** 
*     Do not share datasets with others
*     Do not post datasets on the Web site
*     Datasets can only be used to work on assignments of MDS6212, M.Sc. in Data Science, CUHK-Shenzhen.

**Datasets:** 
*     MDS6212 Week 3 Stock returns.xlsx
*     MDS6212 Week 3 Online sales.xlsx

In [1]:
!ls

'MDS6212 Week 3 Assignment.docx'    'MDS6212 Week 3 Stock returns.xlsx'
'MDS6212 Week 3 Online sales.xlsx'   MDS6212Assignment3.ipynb


In [1]:
import numpy as np
import pandas as pd

## Q1

1) Using the data set of stock returns, sort stocks into quintiles by size every quarter, hold stocks over the quarter, and calculate monthly portfolio returns

In [3]:
Stock_returns = pd.read_excel('MDS6212 Week 3 Stock returns.xlsx', sheet_name='data')

In [4]:
Stock_returns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87432 entries, 0 to 87431
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   stkcd       87432 non-null  int64  
 1   stknme      87432 non-null  object 
 2   conme       87432 non-null  object 
 3   month       87432 non-null  object 
 4   retrf       87432 non-null  float64
 5   mktrf       87432 non-null  float64
 6   smb         87432 non-null  float64
 7   hml         87432 non-null  float64
 8   umd         87432 non-null  float64
 9   size        87432 non-null  float64
 10  bm          87432 non-null  float64
 11  return12    87432 non-null  float64
 12  roa         87432 non-null  float64
 13  lev         87432 non-null  float64
 14  ppe         87432 non-null  float64
 15  intang      87432 non-null  float64
 16  numanalyst  87432 non-null  int64  
 17  instown     87432 non-null  float64
 18  mv          87432 non-null  int64  
dtypes: float64(13), int64(3),

In [5]:
Stock_returns.head()

,stkcd,stknme,conme,month,retrf,mktrf,smb,hml,umd,size,bm,return12,roa,lev,ppe,intang,numanalyst,instown,mv
0,9,中国宝安,中国宝安集团股份有限公司,2013-07,9.0756,1.8833,5.1896,-0.1922,6.1829,23.10871,0.438860,13.37412,0.007949,0.430605,0.086185,0.031575,0,6.4646,11874173952
1,9,中国宝安,中国宝安集团股份有限公司,2013-08,-5.1363,4.5833,5.9407,0.0976,-5.5041,23.10871,0.438860,13.37412,0.007949,0.430605,0.086185,0.031575,0,6.4646,11874173952
2,9,中国宝安,中国宝安集团股份有限公司,2013-09,3.5840,3.3833,0.5848,2.8554,8.8693,23.10871,0.438860,13.37412,0.007949,0.430605,0.086185,0.031575,0,6.4646,11874173952
3,9,中国宝安,中国宝安集团股份有限公司,2014-01,-1.0932,-3.6167,5.6921,1.1126,8.9357,23.28204,0.394303,46.59159,0.025973,0.432246,0.098660,0.029798,0,5.6741,11731143680
4,9,中国宝安,中国宝安集团股份有限公司,2014-02,19.0704,0.9833,3.8899,0.6902,-1.8764,23.28204,0.394303,46.59159,0.025973,0.432246,0.098660,0.029798,0,5.6741,11731143680


In [10]:
Stock_returns['month'] = pd.to_datetime(Stock_returns['month'], format='%Y-%m') #.dt.strftime('%Y-%m')
#groupby
Stock_returns['size'].groupby([Stock_returns['month'].dt.year, Stock_returns['month'].dt.month]).quantile(0.8)

month  month
2012   4        22.656796
       5        22.656796
       6        22.656796
       7        22.622054
       8        22.622054
                  ...    
2019   8        23.168880
       9        23.168880
       10       23.050434
       11       23.050434
       12       23.050434
Name: size, Length: 93, dtype: float64

In [13]:
# Groupby quarterly
Groupby_quarterly = Stock_returns.groupby(Stock_returns['month'].dt.to_period('Q'))
# Try to get the key of 2012Q2
Group_Keys = list(Groupby_quarterly.groups)

In [30]:
for key, group in Groupby_quarterly:
    print(key, ': the quintiles size are', group['size'].quantile(0.2),
          group['size'].quantile(0.4), group['size'].quantile(0.6),
          group['size'].quantile(0.8), group['size'].quantile(1), 'respectivly.')

    First = group[group['size'] <= group['size'].quantile(0.2)].reset_index()
    Second = group[(group['size'] <= group['size'].quantile(0.4)) & (
        group['size'] > group['size'].quantile(0.2))].reset_index()
    Third = group[(group['size'] <= group['size'].quantile(0.6)) &
                  (group['size'] > group['size'].quantile(0.4))].reset_index()
    Fourth = group[(group['size'] <= group['size'].quantile(0.8)) & (
        group['size'] > group['size'].quantile(0.6))].reset_index()
    Fifth = group[(group['size'] <= group['size'].quantile(1)) &
                  (group['size'] > group['size'].quantile(0.8))].reset_index()

    First['weight'] = First['size'] / First['size'].sum()
    Second['weight'] = Second['size'] / Second['size'].sum()
    Third['weight'] = Third['size'] / Third['size'].sum()
    Fourth['weight'] = Fourth['size'] / Fourth['size'].sum()
    Fifth['weight'] = Fifth['size'] / Fifth['size'].sum()

    Portfolio_First = First['weight'].dot(First['retrf'])
    Portfolio_Second = Second['weight'].dot(Second['retrf'])
    Portfolio_Third = Third['weight'].dot(Third['retrf'])
    Portfolio_Fourth = Fourth['weight'].dot(Fourth['retrf'])
    Portfolio_Fifth = Fifth['weight'].dot(Fifth['retrf'])

    print("###########################################", key,
          "###############################################")
    print(
        "After sort stocks into quintiles by size every quarter", '\n', "the monthly portfolio returns of:",
        '\n', "Fisrt quintile is", Portfolio_First, '\n',
        "Second quintile is", Portfolio_Second, '\n', "Third quintile is",
        Portfolio_Third, '\n', "Fourth quintile is", Portfolio_Fourth, '\n',
        "Fifth quintile is", Portfolio_Fifth)
    print("###########################################", key,
          "###############################################")
#     break

2012Q2 : the quintiles size are 21.32591 21.6943 22.04578 22.66286 28.08662 respectivly.
########################################### 2012Q2 ###############################################
After sort stocks into quintiles by size every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -0.36322587894482666 
 Second quintile is 0.6613903439105191 
 Third quintile is 1.4811051769674752 
 Fourth quintile is 0.02593046833750301 
 Fifth quintile is 1.0503672619594473
########################################### 2012Q2 ###############################################
2012Q3 : the quintiles size are 21.271824000000002 21.64703 22.01655 22.623397999999998 28.08148 respectivly.
########################################### 2012Q3 ###############################################
After sort stocks into quintiles by size every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -1.5043196709387954 
 Second quintile is -1.849468786882544 
 Third quintile is -2.7160700321170905 
 

2016Q1 : the quintiles size are 22.0472 22.351330000000004 22.75771 23.31211 27.91817 respectivly.
########################################### 2016Q1 ###############################################
After sort stocks into quintiles by size every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -3.245958861777424 
 Second quintile is -4.060979844888145 
 Third quintile is -5.0100094889799145 
 Fourth quintile is -5.222749429342516 
 Fifth quintile is -4.4717445618634075
########################################### 2016Q1 ###############################################
2016Q2 : the quintiles size are 22.52377 22.861760000000004 23.17243 23.64811 27.93264 respectivly.
########################################### 2016Q2 ###############################################
After sort stocks into quintiles by size every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is 2.6239937351173674 
 Second quintile is 2.986462907158132 
 Third quintile is 2.890585019155251 
 Fourt

## Q2

2) Using the data set of stock returns, sort stocks into quintiles by institutional ownership every quarter, hold stocks over the quarter, and calculate monthly portfolio returns

In [31]:
for key, group in Groupby_quarterly:
    print(key, ': the quintiles instown are', group['instown'].quantile(0.2),
          group['instown'].quantile(0.4), group['instown'].quantile(0.6),
          group['instown'].quantile(0.8), group['instown'].quantile(1), 'respectivly.')

    First = group[group['instown'] <= group['instown'].quantile(0.2)].reset_index()
    Second = group[(group['instown'] <= group['instown'].quantile(0.4)) & (
        group['instown'] > group['instown'].quantile(0.2))].reset_index()
    Third = group[(group['instown'] <= group['instown'].quantile(0.6)) &
                  (group['instown'] > group['instown'].quantile(0.4))].reset_index()
    Fourth = group[(group['instown'] <= group['instown'].quantile(0.8)) & (
        group['instown'] > group['instown'].quantile(0.6))].reset_index()
    Fifth = group[(group['instown'] <= group['instown'].quantile(1)) &
                  (group['instown'] > group['instown'].quantile(0.8))].reset_index()

    First['weight'] = First['instown'] / First['instown'].sum()
    Second['weight'] = Second['instown'] / Second['instown'].sum()
    Third['weight'] = Third['instown'] / Third['instown'].sum()
    Fourth['weight'] = Fourth['instown'] / Fourth['instown'].sum()
    Fifth['weight'] = Fifth['instown'] / Fifth['instown'].sum()

    Portfolio_First = First['weight'].dot(First['retrf'])
    Portfolio_Second = Second['weight'].dot(Second['retrf'])
    Portfolio_Third = Third['weight'].dot(Third['retrf'])
    Portfolio_Fourth = Fourth['weight'].dot(Fourth['retrf'])
    Portfolio_Fifth = Fifth['weight'].dot(Fifth['retrf'])

    print("###########################################", key,
          "###############################################")
    print(
        "After sort stocks into quintiles by instown every quarter", '\n', "the monthly portfolio returns of:",
        '\n', "Fisrt quintile is", Portfolio_First, '\n',
        "Second quintile is", Portfolio_Second, '\n', "Third quintile is",
        Portfolio_Third, '\n', "Fourth quintile is", Portfolio_Fourth, '\n',
        "Fifth quintile is", Portfolio_Fifth)
    print("###########################################", key,
          "###############################################")
#     break

2012Q2 : the quintiles instown are 0.804 3.4267 7.4483 16.2503 59.0109 respectivly.
########################################### 2012Q2 ###############################################
After sort stocks into quintiles by instown every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -1.1244013846816687 
 Second quintile is 0.16156463514574418 
 Third quintile is 0.2862945716186326 
 Fourth quintile is 0.6191326176034919 
 Fifth quintile is 2.1793811515397965
########################################### 2012Q2 ###############################################
2012Q3 : the quintiles instown are 0.5764999999999996 3.9008000000000003 9.4383 20.47526 74.9974 respectivly.
########################################### 2012Q3 ###############################################
After sort stocks into quintiles by instown every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -3.9726258120625824 
 Second quintile is -3.1672300425318722 
 Third quintile is -2.9547563095383174 


########################################### 2016Q1 ###############################################
After sort stocks into quintiles by instown every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is -4.3889591991153845 
 Second quintile is -4.989297390207808 
 Third quintile is -3.755717702274933 
 Fourth quintile is -4.041534083367884 
 Fifth quintile is -5.377097915333136
########################################### 2016Q1 ###############################################
2016Q2 : the quintiles instown are 0.8480999999999999 2.6699 5.6457 10.8335 84.87659 respectivly.
########################################### 2016Q2 ###############################################
After sort stocks into quintiles by instown every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is 0.45679383646400473 
 Second quintile is 2.1095593753124278 
 Third quintile is 2.335554277470924 
 Fourth quintile is 3.243786799335456 
 Fifth quintile is 2.2043323960542374
#######################

########################################### 2019Q4 ###############################################
After sort stocks into quintiles by instown every quarter 
 the monthly portfolio returns of: 
 Fisrt quintile is 0.9070971350102763 
 Second quintile is 1.818466065759386 
 Third quintile is 2.3743207842261786 
 Fourth quintile is 2.562099560452052 
 Fifth quintile is 2.886820546108633
########################################### 2019Q4 ###############################################


## Q3

3) Using the data set of stock returns, perform panel regression, and regress stock returns on firm characteristics such as size, book-to-market ratio, return12, roa, leverage, ppe, intang, number of analysts, institutional ownership, controlling for or not for firm and year-month fixed effects. Cluster standard errors by firm and year-month (double clustering)

## Q4

4) Using the data set of Online sales, aggregate monthly online sales over quarters, download reported quarterly total sales from CSMAR, and plot figures including both online sales and reported quarterly sales.